In [3]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform
import math
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from time import sleep
from tqdm import trange
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import math as mt
from numpy.linalg import inv, det
from matplotlib import pyplot as plt
%matplotlib inline

## Read data

In [4]:
mfeat = pd.read_csv("mfeat/mfeat-fac", delimiter="\0", sep="\0", header=None )
mfeatFou = pd.read_csv("mfeat/mfeat-fou", delimiter="\0", sep="\0", header=None )
mfeatka = pd.read_csv("mfeat/mfeat-kar", delimiter="\0", sep="\0", header=None )

y = [0 for x in range(0, 200)]
y += [1 for x in range(200, 400)]
y += [2 for x in range(400, 600)]
y += [3 for x in range(600, 800)]
y += [4 for x in range(800, 1000)]
y += [5 for x in range(1000, 1200)]
y += [6 for x in range(1200, 1400)]
y += [7 for x in range(1400, 1600)]
y += [8 for x in range(1600, 1800)]
y += [9 for x in range(1800, 2000)]

mfeat

### Data preprocessing

In [5]:
#Retirando os espaços. Transformando da elemento em uma lista com valores númericos, Atribuindo isso a uma 
# lista com tudo processado e normaliza
def preprocessing(data):
    new_data = []
    for ex in data:
        ex = ex[0].split(" ")
        exemplos1 = []
        for element in ex:
            if element != "":
                exemplos1.append(element)
        new_data.append(exemplos1) 
    
    ## Normaliza
    new_data = np.array(new_data).astype(float)
    scaler = StandardScaler()
    scaler.fit(new_data)
    new_data = scaler.transform(new_data)
    return new_data
   


In [6]:
mfeat =  preprocessing(mfeat.values)
mfeatFou =  preprocessing(mfeatFou.values)
mfeatKa =  preprocessing(mfeatka.values)

# Dataset (Train and test)

In [110]:
X_train_mfeat, X_test_mfeat, y_train_mfeat, y_test_mefeat =  train_test_split(mfeat,y, test_size=0.99, random_state=42)
X_train_mfeatFou, X_test_mfeatFou, y_train_mfeatFou, y_test_mefeatFou =  train_test_split(mfeatFou,y, test_size=0.99, random_state=42)
X_train_mfeatKa, X_test_mfeatKa, y_train_mfeatKa, y_test_mefeatKa =  train_test_split(mfeatKa,y, test_size=0.99, random_state=42)

# Euclidean matrix 

In [63]:
matrix_euclian_mfeat = squareform(pdist(mfeat, metric='euclidean'))
matrix_euclian_mfeatFou = squareform(pdist(mfeatFou, metric='euclidean'))
matrix_euclian_mfeatKa = squareform(pdist(mfeatKa, metric='euclidean'))

# Bayes

In [64]:

def calc_density(x, priori, mean, sigma, classes):
    apriori = priori
    mean = mean
    sigma = sigma
    classes = classes
    x = x
    densities = []

    for c in range(0,classes):


        ##determinant
        inver = inv(np.identity(x.shape[0]) * sigma[c])
        determinant= det(inver)

        part_one_equation = mt.pow(2*mt.pi, -x.shape[0]/2)*mt.pow(determinant,0.5)

        ## values for exp calculation
        value1_exp = ((x -mean[c]).T)
        value1_exp = np.dot(value1_exp, inver)
      
        value2_exp = (x-mean[c])

        ##calc  exp
        exp = np.exp(-0.5*(np.dot(value1_exp, value2_exp)))

        #Result conditional x priori

        rest = part_one_equation*exp

        densities.append(rest*priori[c])
        

    return densities

In [157]:

def densities_views():
    nb = GaussianNB()
    nb.fit(X_test_mfeat, y_test_mefeat);


    priori = nb.class_prior_
    sigma = nb.sigma_
    mean = nb.theta_
    classes = nb.classes_
    x = X_test_mfeat

    densities_mfeat= []
    for x in X_test_mfeat:
        densities_mfeat.append(calc_density(x, priori, mean, sigma, 10))


    nb = GaussianNB()
    nb.fit(X_test_mfeatFou, y_test_mefeatFou);


    priori = nb.class_prior_
    sigma = nb.sigma_
    mean = nb.theta_
    classes = nb.classes_
    x = X_test_mfeatFou

    densities_mfeatFou= []
    for x in X_test_mfeatFou:
        densities_mfeatFou.append(calc_density(x, priori, mean, sigma, 10))



    nb = GaussianNB()
    nb.fit(X_test_mfeatKa, y_test_mefeatKa);
    priori = nb.class_prior_
    sigma = nb.sigma_
    mean = nb.theta_
    classes = nb.classes_
    x = X_test_mfeatKa

    densities_mfeatKa= []
    for x in X_test_mfeatKa:
        densities_mfeatKa.append(calc_density(x, priori, mean, sigma, 3))
        
        
    return densities_mfeat,densities_mfeatFou, densities_mfeatKa
    
    

In [158]:
densities_view1, densities_view2, densities_view3 = densities_views()

In [159]:
evidence = densities_view1 + densities_view2+ densities_view3

In [160]:
evidence = np.array(evidence).sum(axis=0)

In [161]:
def posteriori(view):
    
    vet_posteriori = []
    view = np.array(view).sum(axis=0)
    for densitie in view:
         vet_posteriori.append(densitie/evidence)   
     
    maximum_likelihood = np.argmax(vet_posteriori)
    
    return vet_posteriori[maximum_likelihood]

In [162]:
posteriori_view1 = posteriori(densities_view1)
posteriori_view2 = posteriori(densities_view2)
posteriori_view3 = posteriori(densities_view2)

/home/rodrigo/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in true_divide
  


In [78]:
def sum_rule(view1, view2, view3, num_classes):
    
    posteriori_final = (view1+view2+view3)/num_classes
    
    return posteriori_final
    

In [86]:
post_final = sum_rule(posteriori_view1, posteriori_view2, posteriori_view3, 10)

In [103]:
post_final

array([8.11264137e-19, 9.95062014e-10, 1.01354319e-09, 5.44446018e-10,
       3.55826557e-08, 7.42186690e-08, 7.47212313e-09, 5.35813906e-14,
       7.26451689e-15, 1.13470053e-11])

In [88]:
X_test_mfeatFou.shape

(1980, 76)

# Utilizando o Naive Bayes para estimar as probabilidades

In [148]:
nb = GaussianNB()
vi1 = nb.fit(X_test_mfeat, y_test_mefeat)

prob1 = nb.predict_proba(X_test_mfeat)
nb = GaussianNB()
vi2 = nb.fit(X_test_mfeatFou, y_test_mefeatFou)

prob2 = nb.predict_proba(X_test_mfeatFou)

nb = GaussianNB()
vi3 = nb.fit(X_test_mfeatKa, y_test_mefeatKa)
prob3 = nb.predict_proba(X_test_mfeatKa)

In [149]:
prob1 = prob1.sum(axis=0)/1980
prob2 = prob2.sum(axis=0)/1980
prob3 = prob3.sum(axis=0)/1980

In [150]:
prob1

array([0.09562214, 0.10926682, 0.10505956, 0.09779887, 0.10412186,
       0.09441851, 0.0978382 , 0.09418074, 0.0975626 , 0.10413071])

In [151]:
prob2

array([0.09352633, 0.13449731, 0.09192114, 0.08960671, 0.09483517,
       0.10065198, 0.08896956, 0.10784068, 0.10238785, 0.09576326])

In [152]:
prob3

array([0.09520867, 0.10576766, 0.10013804, 0.1001981 , 0.10675518,
       0.09774587, 0.09468783, 0.0917777 , 0.10140092, 0.10632003])

In [155]:
prob_rule_sum = (prob1+prob2+prob3)/3

In [156]:
prob_rule_sum

array([0.09478571, 0.1165106 , 0.09903958, 0.09586789, 0.10190407,
       0.09760546, 0.09383186, 0.09793304, 0.10045046, 0.10207133])